In [1]:
%load_ext autoreload
%autoreload 2

from owlready2 import *
import owlready2
print(owlready2.VERSION)

import pandas as pd
import glob, os, hashlib
import requests, json
from datetime import datetime

import random

def NOW():
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    return "Current Time = "+ str(current_time)

from dotenv import load_dotenv
load_dotenv('.env')

URL = os.getenv("KG_URL_FCT")
TOK3N = os.getenv("KG_TOKEN")

NBCAP = 5000

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


0.40


In [2]:
# @TODO: need to review classification to avoid Nan and duplicates

In [3]:
import importlib.util
import sys

spec = importlib.util.spec_from_file_location("llm", "../../../utils/llm.py")
h = importlib.util.module_from_spec(spec)
sys.modules["llm"] = h
spec.loader.exec_module(h)

spec = importlib.util.spec_from_file_location("rdfutils", "../../../utils/rdfutils.py")
u = importlib.util.module_from_spec(spec)
sys.modules["rdfutils"] = u
spec.loader.exec_module(u)

In [4]:
onto = get_ontology("./WIP.owl").load()
u.cOnto(onto)


# Overview

*  _PBNThing_  --  87887 instances.
*  _BenefReturn_  --  1606 instances.
*  _Benef_  --  1409 instances.
*  _Article_  --  393 instances.
*  _Risk_  --  21167 instances.
*  _ISO_Scale_  --  5 instances.
*  _RiskHealth_  --  14 instances.
*  _RiskType_  --  22 instances.
*  _Stakeholder_  --  16129 instances.
*  _Stakeholder_Type_  --  3 instances.
*  _Technology_  --  23143 instances.
*  _ISO_Impact_  --  24 instances.
*  _ISO_Purpose_  --  18 instances.
*  _StakeholderGroup_  --  18 instances.
*  _StakeholderSubgroup_  --  169 instances.
*  _TechGroup_  --  15 instances.
*  _TechSubgroup_  --  127 instances.
*  _Mitigation_  --  22953 instances.
*  _BP_Enabler_  --  33 instances.
*  _BP_Transmission_  --  12 instances.
*  _Blueprint_  --  74 instances.
*  _BP_Scale_  --  3 instances.
*  _BP_Phase_  --  4 instances.
*  _BP_Permanent_  --  4 instances.
*  _BP_Type_  --  2 instances.
*  _BP_Intervention_  --  46 instances.
*  _PBNCategory_  --  330 instances.
*  _RiskGroup_

# Adding classifications

## Techs


In [5]:
TECHS = [x.label[0] for x in onto.Technology.instances() if len(x.label) >= 1]
len(TECHS)

23143

In [6]:
def classify_technology(CategoriesOfTechs):
    return [
        {
        "name": "classify_technology",
        "description": "Function used to classify technologies from a high level perspective.",
        "parameters": {
            "type": "object",
            "properties": {
                "sorted_technology": {
                    "type": 'array',
                    "items": {
                        "type": 'object',
                        "description": "A type of technology ",
                        "properties": {
                            "name" :{
                                "type": 'string', 
                                "description": 'The technology being considered.'
                            },
                            "category" :{
                                "type": 'string', 
                                "enum":CategoriesOfTechs,
                                "description": 'The category of technology that matches the technology being considered. '
                            },
                            "confidence" :{
                                "type": 'string',  
                                "enum":["High","Medium-high","Medium","Low"],
                                "description": 'Confidence that this is the correct technology category.'
                            }                            
                        },
                        "required": ["name",'category',"confidence"],
                    }
                },
            },
            "required": ["sorted_technology"],
        },
    }
]

In [7]:
DICT = {}
DICT["Air Control"] = ['Air Filtering',
 'Indoor Environment',
 'HVAC Systems',
 'Air Quality Monitoring',
 'Air Purification',
 'Ventilation',
 'HEPA Filters',
 'UV-C Disinfection']
DICT["Data Analytics"] = ['Machine Learning and AI Algorithms', 'Real-Time Data Tracking and Monitoring', 'Data Visualization and Modeling', 'Cloud-Based Data Sharing and Analysis', 'Advanced Statistical Modeling and Analysis', 'Social Media Monitoring and Analysis', 'Predictive Analytics and Modeling Tools',  'Information Dissemination Platforms']
DICT["Infrastructure Management"] = ['Sanitation and hygiene facilities', 'Ventilation and air quality systems', 'Geographic information and mapping technology', 'Healthcare infrastructure and telecommunication networks', 'Energy-efficient technologies and smart grid systems', 'Remote work tools and workforce management technologies', 'Logistics and inventory management systems', 'Capacity-building and infrastructure development tools', 'Disaster management and resilience planning technologies']
DICT["Communication"] = ['Communication and Collaboration Platforms', 'Public Awareness Campaign Tools', 'Video Conferencing and Collaboration Software', 'Social Media Platforms', 'Virtual Communication and Collaboration Technologies', 'Digital Signage and Messaging Systems', 'Multilingual Communication and Outreach Tools', 'Online Public Engagement Platforms', 'Remote Work and Telecommuting Technologies', 'Mobile App Notification Systems']
DICT["Sensors and Monitoring"] = ['Monitoring solutions', 'IoT devices', 'Surveillance systems', 'Environmental sensors', 'Sensor technologies', 'Remote monitoring tools', 'Crowd monitoring technology', 'Data collection systems', 'Real-time tracking systems']
DICT["Healthcare"] = ['Diagnostic Testing', 'Sequencing and Genomic Analysis', 'Analytical Chemistry', 'Physical Health Monitoring', 'Wellbeing and Mental Health', 'Social Distancing Technologies', 'Vaccination Development and Distribution', 'Personal Hygiene and Sanitation', 'Personal Protective Equipment (PPE)', 'Remote Healthcare and Telemedicine']
DICT["Urban Planning"] = ['Community Management Technologies', 'Spatial Planning and Layout Tools', 'Green Infrastructure Management', 'Crowd Control Solutions', 'Smart City Technologies', 'Urban Planning Solutions', 'Resilience Planning Frameworks', 'Urban Design and Mapping Tools', 'Environmental Conservation Technologies']
DICT["Digital and Information Management"] = ['Data Management', 'Web Development', 'Network Security', 'Cloud Services', 'Mobile Applications', 'Knowledge Management', 'Ontology Tools', 'Data Governance', 'Digital Infrastructure', 'Information Systems']
DICT["Building design"] = ['Building design', 'Ventilation systems', 'Home design and architecture', 'Sustainable design', 'Soundproofing technology', 'Green building strategies', 'Space optimization technologies', 'Architectural design software', 'Adaptive construction methods', 'Virtual planning and design tools']
DICT["Privacy Enhancing Technologies"] = ['Encryption and authentication', 'Biometrics and identification', 'Face and voice recognition', 'Data encryption and security', 'Cybersecurity and access control', 'Quantum cryptography', 'Ethical guidelines and advocacy', 'Fact-checking and information verification', 'Blockchain for data privacy and transparency']
DICT["Building Management"] = ['Access control and security technology', 'Lighting and daylighting systems', 'Maintenance and landscaping equipment', 'Door-locking and automatic systems', 'Operation of HVAC and ventilation systems', 'Building management and automation systems', 'Industry standard compliance tools', 'Safety and hazard detection systems']
DICT["Cleaning"] = ['Automated sanitation equipment', 'Advanced disinfection systems', 'Surface cleaning', 'Waste management technologies group', 'Sanitization equipment group', 'Disinfectant formulation', 'Self-cleaning technologies', 'Hygiene maintenance tools']
DICT["Supply-chain"] = ['Diversification and suppliers', 'Supply chain and risk management',  'Manufacturing of PPE', 'Digital supply chain management', 'Contactless delivery']
DICT["Water Control"] = ['Water and Wastewater Management', 'Water Treatment Systems', 'Water Quality Monitoring', 'Water Disinfection Solutions',  'Advanced Filtration Technologies', 'Sewage Treatment Facilities', 'Water Recycling and Conservation']
DICT["Waste Management"] = ['Recycling and waste handling technologies', 'Advanced waste sorting and processing systems', 'Smart waste handling and monitoring technologies', 'Contactless recycling and waste disposal solutions', 'Automated waste collection and segregation technologies', 'Sanitization and sterilization technologies for waste', 'Improved waste storage and disposal systems']

CategoriesOfTechs = list(DICT.keys())

### Doing Level 1 of technologies

In [8]:
RES = []
NChunks = 50
#Randomizing list
if os.path.isfile("../../support/newtechs_classified_lv1.parquet.gzip"):
    GOOD = pd.read_parquet("../../support/newtechs_classified_lv1.parquet.gzip")
    GOOD = GOOD[GOOD.name.isin(TECHS)]
    alreadythere = GOOD["name"].unique()
    newRisks = [x for x in TECHS if x not in alreadythere]
    CATS_FWK = [GOOD]
else:
    newRisks = TECHS
    CATS_FWK = []

if len(newRisks):
    newRisks = [str(x) for x in newRisks]
    NB = len(newRisks)//NChunks + 1

    for k in range(NB):
        try:
            QUESTION =  "The technologies are:\n\n* "+"\n* ".join(newRisks[(k*NChunks):(k+1)*NChunks]) 
            prefix ="technologies_"
            overwrite = False
            REQ = {
                "context": "Out the following list, classify the technologies from a high level perspective.\n\n",
                "question": QUESTION,
                "functions": classify_technology(CategoriesOfTechs),
                "token": TOK3N,
                "overwrite": overwrite,
                "source": "local-evalTechsT3.5",
                "seed" : ""
            }

            H = h.hashme(REQ["context"]+REQ["question"])
            cached = "cache/"+prefix+"_"+H+".json"
            if not os.path.isfile(cached) or overwrite:
                x = requests.post(URL+"fct/", json = REQ)
                answer = json.loads(x.text)["messages"][-1]
                h.svt(cached,answer)
            else:
                answer = h.ldt(cached)
            d = pd.DataFrame(json.loads(answer["function_call"]["arguments"])["sorted_technology"])
            d = d [d["category"].isin(CategoriesOfTechs)]
            CATS_FWK.append(d)
            DFC = pd.concat(CATS_FWK).reset_index(drop=True)
            DFC = DFC.sort_values(by=["name","category"],ascending=True).drop_duplicates(subset=["name"]).reset_index(drop=True)
            DFC = DFC.sort_values(by=["name","subcategory"],ascending=True).drop_duplicates(subset=["name"]).reset_index(drop=True)
            #DFC.to_parquet("../../support/newtechs_classified_lv1.parquet.gzip",compression="gzip")
            print("Done:",k)
        except:
            print("error")
            pass

Done: 0
Done: 1


### Doing Level 2 of technologies

In [9]:
DFC = pd.read_parquet("../../support/newtechs_classified_lv1.parquet.gzip")
print(len(DFC))
DFC.head(2)

23085


,name,category,confidence,co```nfidence,&confidence,subcategory,confidence_lv2
0,-80°c freezers,Healthcare,High,None,None,Vaccination Development and Distribution,High
1,14-day quarantine,Healthcare,High,None,None,Social Distancing Technologies,High


In [10]:
DFC = pd.read_parquet("../../support/newtechs_classified_lv1.parquet.gzip")
CATs = list(DFC.category.unique())
if "subcategory" not in DFC.columns:
    DFC["subcategory"] = None
if "confidence_lv2" not in DFC.columns:
    DFC["confidence_lv2"] = None
for CAT in CATs:
    print("Doing",CAT)
    df = DFC[DFC.category == CAT]
    DONE = list(df[~df.subcategory.isna()].name.unique())
    df = df[~df.name.isin(DONE)]
    subtechs = DICT[CAT] # classify_technology(CategoriesOfTechs)

    newRisks = list(df.name)
    RES = []
    NChunks = 50

    if len(newRisks):
        newRisks = [str(x) for x in newRisks]
        NB = len(newRisks)//NChunks + 1

        for k in range(NB)[:NBCAP]:
            try:
                QUESTION =  "The technologies are:\n\n* "+"\n* ".join(newRisks[(k*NChunks):(k+1)*NChunks]) 
                prefix ="technologies_"
                overwrite = False
                REQ = {
                    "context": "Out the following list, classify the technologies from a high level perspective.\n\n",
                    "question": QUESTION,
                    "functions": classify_technology(subtechs),
                    "token": TOK3N,
                    "overwrite": overwrite,
                    "source": "local-evalTechsT3.5",
                    "seed" : ""
                }

                H = h.hashme(REQ["context"]+REQ["question"])
                cached = "cache/"+prefix+"_"+H+".json"
                if not os.path.isfile(cached) or overwrite:
                    x = requests.post(URL+"fct/", json = REQ)
                    answer = json.loads(x.text)["messages"][-1]
                    h.svt(cached,answer)
                else:
                    answer = h.ldt(cached)
                d = pd.DataFrame(json.loads(answer["function_call"]["arguments"])["sorted_technology"])
                d = d [d["category"].isin(subtechs)]
                d.columns = ["name","subcategory","confidence_lv2"]
                for ix, row in d.iterrows():
                    DFC.loc[DFC.name == row["name"], "subcategory"] = row["subcategory"]
                    DFC.loc[DFC.name == row["name"], "confidence_lv2"] = row["confidence_lv2"]
                DFC = DFC.sort_values(by=["name","category"],ascending=True).drop_duplicates(subset=["name"]).reset_index(drop=True)
                DFC.to_parquet("../../support/newtechs_classified_lv1.parquet.gzip",compression="gzip")
                #DFC = DFC.sort_values(by=["name","subcategory"],ascending=True).drop_duplicates(subset=["name"]).reset_index(drop=True)
                print("Done:",k)
            except:
                print("error")
                pass

Doing Healthcare
Done: 0
Done: 1
Doing Building Management
Done: 0
error
Doing Building design
Done: 0
Done: 1
Doing Digital and Information Management
Done: 0
Done: 1
Done: 2
Doing Urban Planning
Done: 0
Done: 1
Doing Infrastructure Management
Done: 0
Doing Supply-chain
Done: 0
Done: 1
Doing Communication
Done: 0
Done: 1
Doing Sensors and Monitoring
Done: 0
Done: 1
Done: 2
Doing Air Control
Done: 0
Doing Water Control
Done: 0
Doing Data Analytics
Done: 0
Done: 1
error
Doing Privacy Enhancing Technologies
Doing Cleaning
Doing Waste Management
Done: 0


### Classification 

In [11]:
import pandas as pd
DFC = pd.read_parquet("../../support/newtechs_classified_lv1.parquet.gzip")
DFC[~DFC.subcategory.isna()].groupby(["category","subcategory"]).name.count().to_frame()

name
category      subcategory                           
Air Control   Advanced disinfection systems        1
              Air Filtering                       80
              Air Purification                   120
              Air Quality Monitoring             143
              Diagnostic Testing                   1
...                                              ...
Water Control Water Disinfection Solutions        26
              Water Quality Monitoring            64
              Water Recycling and Conservation    17
              Water Treatment Systems             58
              Water and Wastewater Management    119

[329 rows x 1 columns]

## Stakeholders

In [12]:
SH = [x.label[0] for x in onto.Stakeholder.instances() if len(x.label) >= 1]
len(SH)

16129

In [13]:
DICT = {'Health Organisations': ['Public health authorities',
  'Occupational health and safety teams',
  'Government health agencies ',
  'International health organizations',
  'Medical and healthcare personnel',
  'Private health organisations',
  'Mental healthcare providers',
  'Veterinary organization'],
 'Research Organisations': ['AI and Data analysis researchers',
  'Research institutions ',
  'Technology developers',
  'Academic institutions',
  'Data repository administrators',
  'Local researchers',
  'Citizen researchers',
  'Scientific communities'],
 'Government institutions': ['City authorities',
  'Government departments ',
  'Policymakers',
  'Diplomatic bodies',
  'Local government',
  'Educational organizations',
  'Political communities',
  'Administrative authorities',
  'Regulatory bodies ',
  'Emergency services and management',
  'International development agencies'],
 'General public': ['Immigrant population',
  'Patients',
  'Travelers',
  'Civil society stakeholders',
  'Recreational and fitness enthusiasts',
  'Visitors in indoor spaces',
  'General population ',
  'Digitally connected individuals',
  'Fragile and/or isolated population',
  'Individual with high health risks',
  'Green spaces users '],
 'Environmental and sustainability organizations': ['Environmental agencies',
  'Conservation organizations',
  'Park management stakeholders',
  'Waste management organizations',
  'Sustainability teams ',
  'Energy and environmental consultants',
  'Green area managers and organizations',
  'Wildlife and ecosystem conservationists',
  'Climate and sustainability research institutions'],
 'Residents and occupants of spaces': ['Homeowners ',
  'Building managers',
  'People with lower income',
  'People in high-contamination areas',
  'Occupants of shared household spaces',
  'Visitors to green spaces',
  'Individuals working from home'],
 'Workers': ['Office workers',
  'Physical workers',
  'Healthcare workers',
  'Real estate agents',
  'Facility maintenance team',
  'AI system developers',
  'Waste management workers',
  'Project managers'],
 'Regulatory bodies': ['Global governing bodies',
  'Trade regulators',
  'Enforcement organizations',
  'Certification system organizations',
  'Standardization bodies',
  'Data governance organizations',
  'Governments and industry regulatory bodies',
  'Security agency',
  'Occupational health and safety regulators',
  'Local regulatory authorities',
  'International trade regulators'],
 'Public Entities': ['Sports-related organizations',
  'Appointed personnel responsible for prevention measures',
  'Education-related entities',
  'Water utilities',
  'Energy Utilities',
  'Tourism organizations ',
  'Culture organizations',
  'Police ',
  'Public communication authorities'],
 'Financial groups': ['Banking institutions',
  'Funding Bodies',
  'Economic analysts and regulatory authorities',
  'International financial institutions',
  'Treasury departments',
  'Small-and medium-sized enterprises',
  'Companies in general',
  'Global markets',
  'Insurances providers',
  'Finance professionals',
  'Social entrepreneurship fund managers',
  'Investors',
  'Social entrepreneurship funds'],
 'Construction and infrastructure industry': ['Building design team',
  'Workers in the construction industry',
  'Construction industry organizations',
  'Contractors ',
  'HVAC equipment providers',
  'Site-supervision workers',
  'Operational staff',
  'Design and construction professionals',
  'Building maintenance and green building professionals',
  'Facility maintenance staff'],
 'Manufacturing and production sector': ['Production staff',
  'Non-medical cechnology manufacturers',
  'Medical technology manufacturers',
  'Food industry companies',
  'International suppliers',
  'Supply chain professionals',
  'Wearable technology industry',
  'Pharmaceutical industry'],
 'Transportation and mobility sector': ['Logistics professionals',
  'Ride-sharing ',
  'City transport authorities',
  'Vehicle users',
  'Safe mobility',
  'Soft mobility',
  'Route optimization specialists',
  'Airports',
  'Ports',
  'Railways',
  'Delivery services',
  'Logistics and transportation managers'],
 'Cities professionals': ['City planning professionals',
  'Urban development authorities',
  'Individual urban planners and designers',
  'Urban planning institutions',
  'Smart city professionals',
  'Sustainable city professionals',
  'International urban institutions',
  'Public space managers'],
 'Retail and service industry': ['Cleaning personnel',
  'Retail staff',
  'Marketing agency',
  'Local businesses',
  'Food distribution',
  'Animal rearing industry',
  'Online retailers',
  'Drive-through staff',
  'Supermarket and health-care facility staff',
  'Parking lot owners',
  'Local fitness businesses'],
 'Energy Sector': ['Energy monitoring system providers',
  'Operators and maintenance staff ',
  'Renewable energy investors',
  'Energy management authorities',
  'Energy communities',
  'Communities reliant on the coal and oil industries',
  'Global energy markets',
  'Local energy markets',
  'Energy grid operators'],
 'Media organizations': ['Local communication experts',
  'Individuals consuming digital content',
  'Media and news entities',
  'Marketing/advertising agencies',
  'Reporter',
  'Social media platform providers',
  'Academic journals',
  'News outlet'],
 'Cities Officials': ['Municipality officials',
  'Public building managers',
  'City authorities ',
  'Public space managers',
  'Local government and urban development authorities',
  'Community leaders',
  'Smart city managers',
  'Other cities departments']}

In [14]:
def classify_SH(CategoriesOfStakeholders):
    return [
        {
        "name": "classify_SH",
        "description": "Function used to classify stakeholders from a high level perspective, based on what group they are most closely belonging to. ",
        "parameters": {
            "type": "object",
            "properties": {
                "sorted_stakeholders": {
                    "type": 'array',
                    "items": {
                        "type": 'object',
                        "description": "A type of stakeholder ",
                        "properties": {
                            "name" :{
                                "type": 'string', 
                                "description": 'The stakeholder being considered.'
                            },
                            "category" :{
                                "type": 'string', 
                                "enum":CategoriesOfStakeholders,
                                "description": 'The category of stakeholder that matches the stakeholder category being considered. '
                            },
                            "confidence" :{
                                "type": 'string',  
                                "enum":["High","Medium-high","Medium","Low"],
                                "description": 'Confidence that this is the correct stakeholder category.'
                            }                            
                        },
                        "required": ["name",'category',"confidence"],
                    }
                },
            },
            "required": ["sorted_stakeholders"],
        },
    }
]

In [15]:
CategoriesOfStakeholders = DICT.keys()
RES = []
NChunks = 50
#Randomizing list
if os.path.isfile("../../support/newstakeholders_classified_lv1.parquet.gzip"):
    GOOD = pd.read_parquet("../../support/newstakeholders_classified_lv1.parquet.gzip")
    alreadythere = GOOD["name"].unique()
    GOOD = GOOD[GOOD.name.isin(SH)]
    newRisks = [x for x in SH if x not in alreadythere]
    CATS_FWK = [GOOD]
else:
    newRisks = SH
    CATS_FWK = []
if len(newRisks):
    NB = len(newRisks)//NChunks + 1
    for k in range(NB):
        try:
            QUESTION =  "The stakeholders are:\n\n* "+"\n* ".join(newRisks[(k*NChunks):(k+1)*NChunks]) 
            prefix ="sh_"
            overwrite = False
            REQ = {
                "context": "Out the following list, classify the stakeholders from a high level perspective.\n\n",
                "question": QUESTION,
                "functions": classify_SH(list(CategoriesOfStakeholders)),
                "token": TOK3N,
                "overwrite": overwrite,
                "source": "local-evalSHT3.5",
                "seed" : ""
            }

            H = h.hashme(REQ["context"]+REQ["question"])
            cached = "cache/"+prefix+"_"+H+".json"
            if not os.path.isfile(cached) or overwrite:
                x = requests.post(URL+"fct/", json = REQ)
                answer = json.loads(x.text)["messages"][-1]
                h.svt(cached,answer)
            else:
                answer = h.ldt(cached)
            d = pd.DataFrame(json.loads(answer["function_call"]["arguments"])["sorted_stakeholders"])
            d = d [d["category"].isin(CategoriesOfStakeholders)]
            CATS_FWK.append(d)
            DFC = pd.concat(CATS_FWK).reset_index(drop=True)
            DFC = DFC.sort_values(by=["name","category"],ascending=True).drop_duplicates(subset=["name"]).reset_index(drop=True)
            DFC.to_parquet("../../support/newstakeholders_classified_lv1.parquet.gzip",compression="gzip")
            #DFC = DFC.sort_values(by=["name","subcategory"],ascending=True).drop_duplicates(subset=["name"]).reset_index(drop=True)
            print("Done:",k)
        except:
            print("error")
            pass
else:
    print("All risks covered")

Done: 0
Done: 1


In [16]:
import pandas as pd
DFC = pd.read_parquet("../../support/newstakeholders_classified_lv1.parquet.gzip")
print(len(DFC),"items listed")
DFC.groupby(["category"]).name.count()

16074 items listed


category
Cities Officials                                   103
Cities professionals                               121
Construction and infrastructure industry           483
Energy Sector                                      112
Environmental and sustainability organizations     629
Financial groups                                   483
General public                                    2529
Government institutions                           1169
Health Organisations                              1894
Manufacturing and production sector                474
Media organizations                                176
Public Entities                                    551
Regulatory bodies                                  326
Research Organisations                            1566
Residents and occupants of spaces                 3253
Retail and service industry                        307
Transportation and mobility sector                 243
Workers                                           1655
N

In [17]:
DFC = pd.read_parquet("../../support/newstakeholders_classified_lv1.parquet.gzip")
CATs = list(DFC.category.unique())
if "subcategory" not in DFC.columns:
    DFC["subcategory"] = None
if "confidence_lv2" not in DFC.columns:
    DFC["confidence_lv2"] = None
for CAT in CATs:
    print("Doing",CAT)
    df = DFC[DFC.category == CAT]
    DONE = list(df[~df.subcategory.isna()].name.unique())
    df = df[~df.name.isin(DONE)]
    subtechs = DICT[CAT] # classify_technology(CategoriesOfTechs)

    newRisks = list(df.name)
    RES = []
    NChunks = 50

    if len(newRisks):
        newRisks = [str(x) for x in newRisks]
        NB = len(newRisks)//NChunks + 1

        for k in range(NB)[:NBCAP]:
            try:
                QUESTION =  "The stakeholders are:\n\n* "+"\n* ".join(newRisks[(k*NChunks):(k+1)*NChunks]) 
                prefix ="technologies_"
                overwrite = False
                REQ = {
                    "context": "Out the following list, classify the stakeholders from a high level perspective.\n\n",
                    "question": QUESTION,
                    "functions": classify_SH(subtechs),
                    "token": TOK3N,
                    "overwrite": overwrite,
                    "source": "local-evalTechsT3.5",
                    "seed" : ""
                }

                H = h.hashme(REQ["context"]+REQ["question"])
                cached = "cache/"+prefix+"_"+H+".json"
                if not os.path.isfile(cached) or overwrite:
                    x = requests.post(URL+"fct/", json = REQ)
                    answer = json.loads(x.text)["messages"][-1]
                    h.svt(cached,answer)
                else:
                    answer = h.ldt(cached)
                d = pd.DataFrame(json.loads(answer["function_call"]["arguments"])["sorted_stakeholders"])
                d = d [d["category"].isin(subtechs)]
                d.columns = ["name","subcategory","confidence_lv2"]
                for ix, row in d.iterrows():
                    DFC.loc[DFC.name == row["name"], "subcategory"] = row["subcategory"]
                    DFC.loc[DFC.name == row["name"], "confidence_lv2"] = row["confidence_lv2"]
                DFC = DFC.sort_values(by=["name","category"],ascending=True).drop_duplicates(subset=["name"]).reset_index(drop=True)
                DFC.to_parquet("../../support/newstakeholders_classified_lv1.parquet.gzip",compression="gzip")
                #DFC = DFC.sort_values(by=["name","subcategory"],ascending=True).drop_duplicates(subset=["name"]).reset_index(drop=True)
                print("Done:",k)
            except:
                print("error")
                pass

Doing General public
Done: 0
Done: 1
Doing Public Entities
Done: 0
Done: 1
Doing Workers
Done: 0
Doing Manufacturing and production sector
Doing Construction and infrastructure industry
Done: 0
Doing Transportation and mobility sector
Doing Research Organisations
Done: 0
Doing Media organizations
Done: 0
Doing Environmental and sustainability organizations
Done: 0
Doing Retail and service industry
Done: 0
Doing Residents and occupants of spaces
Done: 0
Doing Regulatory bodies
Done: 0
Doing Health Organisations
Done: 0
Doing Government institutions
Done: 0
Done: 1
Doing Financial groups
Done: 0
Doing Cities professionals
error
Doing Energy Sector
Done: 0
Doing Cities Officials


## Risks


In [18]:
RISKS = [x.label[0] for x in onto.Risk.instances() if len(x.label) >= 1]
len(RISKS)

21166

In [19]:
DICT = {'Political risks': ['Trust and transparency challenges',
  'Globalization challenges',
  'Lack of global governance and coordination',
  'Weak local decision-making mechanisms',
  'Lack of coordination and consistency in mitigation measures',
  'Ineffectiveness of interventions',
  'Bioterrorism threats',
  'Global coordination challenges',
  'Lack of transparency and clarity'],
 'Social risks': ['Disinformation and Misinformation',
  'Social and Psychological Effects ',
  'Social Justice Concerns',
  'Communication and Messaging',
  'Access to infrastructure  ',
  'Physical activity and mobility',
  'Misinformation and Media Risks',
  'Cultural and Lifestyle Risks',
  'Vulnerability risks',
  'Occupational and Employment Risks',
  'Mental and Emotional Wellbeing',
  'Health Behaviors and Compliance'],
 'Infrastructure risks': ['Inadequate remote work setup for employees',
  'Improper architectural designs',
  'Inadequate mobility schemes',
  'Poor urban planning',
  'Infrastructure maintenance risk ',
  'Lack of emphasis on air quality ',
  'Lack of emphasis on water quality ',
  'Insufficient adoption of health-focused building standards',
  'Insufficient adoption of health-focused urban standards',
  'Inadequate hospital design ',
  'Inadequate adaptability to emerging problems  ',
  'Disruptions from outbreaks and other major external events'],
 'Public health risks': ['Failure to implement preventive measures',
  'Misinterpretation of data and results',
  'Lack of access to health services',
  'Public health messaging and education',
  'Lack of effective medical solutions',
  'Healthcare resource shortage and management',
  'Workplace safety and occupational exposure',
  'Poor access to Diagnostics and Treatment',
  'Poor vector-borne diseases control'],
 'Data Privacy and Security risks': ['Data Security risks',
  'Privacy risks',
  'Cybersecurity Threats',
  'Inaccurate or Incomplete Data',
  'Misuse of Personal Data',
  'Reliability and Quality of Data',
  'Surveillance and Monitoring Privacy Concerns',
  'Limited Accessibility and Data Equity',
  'Transparency and Accountability Issues',
  'Potential Misinformation and False Data'],
 'Economic risks': ['Supply chain disruptions',
  'Pandemic impact on markets',
  'Lower investment',
  'Lower employment',
  'Discontinuity of services',
  'Global recession and economic instability',
  'Construction industry impacts',
  'Resource and capability risks',
  'Workforce and labor shortages',
  'Financial strains and liquidity crises',
  'Food supply chain disruptions '],
 'Legal risks': ['Compliance and Contractual',
  'Data and Research Integrity',
  'Intellectual Property',
  'Workplace Safety and Health',
  'Privacy and Civil Liberties',
  'Fraud and Misconduct',
  'Healthcare Legislation',
  'Building Codes and Certification',
  'Technology Deployment and Surveillance'],
 'Environmental risks': ['Climate change',
  'Pollution increase',
  'Environmental health risks',
  'Air quality and pollution effects',
  'Urbanization and habitat loss',
  'Climate change effects',
  'Wastewater and sanitation risks',
  'Biodiversity and ecological impacts',
  'Energy consumption and sustainability issues'],
 'Technological risks': ['Data privacy and security risks',
  'Healthcare and medical risks',
  'Infrastructure and connectivity risks',
  'Modeling and simulation risks',
  'Modeling and prediction risks',
  'Artificial intelligence and machine learning risks',
  'Inadequate ventilation and air circulation',
  'Operational and implementation risks ',
  'Innovation and adoption risks',
  'Healthcare infrastructure and technology adoption risks ']}

In [20]:
CategoriesOfRisks = list(DICT.keys())

In [21]:
def classify_risks_PESTLE(CategoriesOfRisks):
    return [
        {
        "name": "classify_risks_PESTLE",
        "description": "Function used to classify risks from a high level perspective, based on what they  impact the most . ",
        "parameters": {
            "type": "object",
            "properties": {
                "sorted_risks": {
                    "type": 'array',
                    "items": {
                        "type": 'object',
                        "description": "A type of risk ",
                        "properties": {
                            "name" :{
                                "type": 'string', 
                                "description": 'The risk being considered.'
                            },
                            "category" :{
                                "type": 'string', 
                                "enum":CategoriesOfRisks,
                                "description": 'The category of risk that matches the risk category being considered. '
                            },
                            "confidence" :{
                                "type": 'string',  
                                "enum":["High","Medium-high","Medium","Low"],
                                "description": 'Confidence that this is the correct risk category.'
                            }                            
                        },
                        "required": ["name",'category',"confidence"],
                    }
                },
            },
            "required": ["sorted_risks"],
        },
    }
]

In [22]:
RES = []
NChunks = 50
#Randomizing list
if os.path.isfile("../../support/newrisks_classified_lv1.parquet.gzip"):
    GOOD = pd.read_parquet("../../support/newrisks_classified_lv1.parquet.gzip")
    alreadythere = GOOD["name"].unique()
    GOOD = GOOD[GOOD.name.isin(RISKS)]
    newRisks = [x for x in RISKS if x not in alreadythere]
else:
    newRisks = RISKS
    
if len(newRisks):
    newRisks = [str(x) for x in newRisks]
    NB = len(newRisks)//NChunks + 1
    CATS_FWK = [GOOD]
    for k in range(NB)[:150]:
        try:
            QUESTION =  "The risks are:\n\n* "+"\n* ".join(newRisks[(k*NChunks):(k+1)*NChunks]) 
            prefix ="risks_"
            overwrite = False
            REQ = {
                "context": "Out the following list, classify the risks from a high level perspective.\n\n",
                "question": QUESTION,
                "functions": classify_risks_PESTLE(CategoriesOfRisks),
                "token": TOK3N,
                "overwrite": overwrite,
                "source": "local-evalrisksT3.5",
                "seed" : ""
            }

            H = h.hashme(REQ["context"]+REQ["question"])
            cached = "cache/"+prefix+"_"+H+".json"
            if not os.path.isfile(cached) or overwrite:
                x = requests.post(URL+"fct/", json = REQ)
                answer = json.loads(x.text)["messages"][-1]
                h.svt(cached,answer)
            else:
                answer = h.ldt(cached)
            d = pd.DataFrame(json.loads(answer["function_call"]["arguments"])["sorted_risks"])
            d = d [d["category"].isin(CategoriesOfRisks)]
            CATS_FWK.append(d)
            DFC = pd.concat(CATS_FWK).reset_index(drop=True)
            DFC = DFC.sort_values(by=["name","category"],ascending=True).drop_duplicates(subset=["name"]).reset_index(drop=True)
            DFC.to_parquet("../../support/newrisks_classified_lv1.parquet.gzip",compression="gzip")
            #DFC = DFC.sort_values(by=["name","subcategory"],ascending=True).drop_duplicates(subset=["name"]).reset_index(drop=True)
            print("Done:",k)
        except:
            print("error")
            pass
else:
    print("All risks covered")

Done: 0
Done: 1
Done: 2


In [23]:
import pandas as pd
DFC = pd.read_parquet("../../support/newrisks_classified_lv1.parquet.gzip")
print(len(DFC),"items listed")
DFC.groupby(["category"]).name.count()

21072 items listed


category
Data Privacy and Security risks     925
Economic risks                     2282
Environmental risks                1630
Infrastructure risks               2948
Legal risks                         211
Political risks                    1149
Public health risks                6364
Social risks                       3478
Technological risks                2085
Name: name, dtype: int64

In [24]:
DFC

,name,category,confidence,subcategory,confidence_lv2
0,180% increase in ipv at a radiology clinic dur...,Social risks,High,Social Justice Concerns,High
1,30% decline in cp revenue projected nationwide,Economic risks,Medium-high,Financial strains and liquidity crises,High
2,30% of hospitalised cases requiring critical care,Public health risks,High,Failure to implement preventive measures,Medium
3,33% of works do not identify pandemic disruptions,Economic risks,High,Pandemic impact on markets,Medium-high
4,35 cohort studies and 69 cross-sectional studi...,Public health risks,High,Workplace safety and occupational exposure,High
...,...,...,...,...,...
21067,Zoonotic infections from wild animals,Public health risks,High,Public health messaging and education,Medium
21068,Zoonotic tick-borne pathogens in urban areas,Public health risks,High,Failure to implement preventive measures,High
21069,Zoonotic transmission,Public health risks,High,Failure to implement preventive measures,Medium
21070,Zoonotic transmission from live wild animal tr...,Public health risks,High,Poor vector-borne diseases control,Medium-high


In [25]:
DFC = pd.read_parquet("../../support/newrisks_classified_lv1.parquet.gzip")
CATs = list(DFC.category.unique())
if "subcategory" not in DFC.columns:
    DFC["subcategory"] = None
if "confidence_lv2" not in DFC.columns:
    DFC["confidence_lv2"] = None

In [26]:
DFC = pd.read_parquet("../../support/newrisks_classified_lv1.parquet.gzip")
CATs = list(DFC.category.unique())
if "subcategory" not in DFC.columns:
    DFC["subcategory"] = None
if "confidence_lv2" not in DFC.columns:
    DFC["confidence_lv2"] = None
for CAT in CATs:
    print("Doing",CAT)
    df = DFC[DFC.category == CAT]
    DONE = list(df[~df.subcategory.isna()].name.unique())
    df = df[~df.name.isin(DONE)]
    subtechs = DICT[CAT] # classify_technology(CategoriesOfTechs)

    newRisks = list(df.name)
    RES = []
    NChunks = 50

    if len(newRisks):
        newRisks = [str(x) for x in newRisks]
        NB = len(newRisks)//NChunks + 1

        for k in range(NB)[:NBCAP]:
            try:
                QUESTION =  "The risks are:\n\n* "+"\n* ".join(newRisks[(k*NChunks):(k+1)*NChunks]) 
                prefix ="technologies_"
                overwrite = False
                REQ = {
                    "context": "Out the following list, classify the risks from a high level perspective.\n\n",
                    "question": QUESTION,
                    "functions": classify_risks_PESTLE(subtechs),
                    "token": TOK3N,
                    "overwrite": overwrite,
                    "source": "local-evalTechsT3.5",
                    "seed" : ""
                }

                H = h.hashme(REQ["context"]+REQ["question"])
                cached = "cache/"+prefix+"_"+H+".json"
                if not os.path.isfile(cached) or overwrite:
                    x = requests.post(URL+"fct/", json = REQ)
                    answer = json.loads(x.text)["messages"][-1]
                    h.svt(cached,answer)
                else:
                    answer = h.ldt(cached)
                d = pd.DataFrame(json.loads(answer["function_call"]["arguments"])["sorted_risks"])
                d = d [d["category"].isin(subtechs)]
                d.columns = ["name","subcategory","confidence_lv2"]
                for ix, row in d.iterrows():
                    DFC.loc[DFC.name == row["name"], "subcategory"] = row["subcategory"]
                    DFC.loc[DFC.name == row["name"], "confidence_lv2"] = row["confidence_lv2"]
                DFC = DFC.sort_values(by=["name","category"],ascending=True).drop_duplicates(subset=["name"]).reset_index(drop=True)
                DFC.to_parquet("../../support/newrisks_classified_lv1.parquet.gzip",compression="gzip") 
                print("Done:",k)
            except:
                print("error")
                pass

Doing Social risks
Done: 0
Doing Economic risks
Done: 0
Doing Public health risks
Done: 0
Done: 1
Done: 2
Doing Technological risks
Done: 0
Done: 1
Done: 2
Doing Infrastructure risks
Done: 0
Done: 1
Doing Political risks
Done: 0
Done: 1
Doing Environmental risks
Done: 0
Doing Data Privacy and Security risks
Done: 0
Doing Legal risks


In [27]:
DFC

,name,category,confidence,subcategory,confidence_lv2
0,180% increase in ipv at a radiology clinic dur...,Social risks,High,Social Justice Concerns,High
1,30% decline in cp revenue projected nationwide,Economic risks,Medium-high,Financial strains and liquidity crises,High
2,30% of hospitalised cases requiring critical care,Public health risks,High,Failure to implement preventive measures,Medium
3,33% of works do not identify pandemic disruptions,Economic risks,High,Pandemic impact on markets,Medium-high
4,35 cohort studies and 69 cross-sectional studi...,Public health risks,High,Workplace safety and occupational exposure,High
...,...,...,...,...,...
21067,Zoonotic infections from wild animals,Public health risks,High,Public health messaging and education,Medium
21068,Zoonotic tick-borne pathogens in urban areas,Public health risks,High,Failure to implement preventive measures,High
21069,Zoonotic transmission,Public health risks,High,Failure to implement preventive measures,Medium
21070,Zoonotic transmission from live wild animal tr...,Public health risks,High,Poor vector-borne diseases control,Medium-high


In [28]:
import pandas as pd
DFC = pd.read_parquet("../../support/newrisks_classified_lv1.parquet.gzip")
print(len(DFC),"items listed")
DFC.groupby(["subcategory"]).name.count()

21072 items listed


subcategory
Access to infrastructure                              255
Air quality and pollution effects                     243
Artificial intelligence and machine learning risks    130
Biodiversity and ecological impacts                   142
Bioterrorism threats                                    3
                                                     ... 
Wastewater and sanitation risks                       138
Weak local decision-making mechanisms                 204
Workforce and labor shortages                          99
Workplace Safety and Health                             7
Workplace safety and occupational exposure            804
Name: name, Length: 91, dtype: int64

In [29]:
import pandas as pd
DFC = pd.read_parquet("../../support/newmitigation_classified_lv1.parquet.gzip")
print(len(DFC),"items listed")
DFC.groupby(["category"]).name.count()

22615 items listed


category
Absorb resource shortages                                             313
Absorb surge in service demande                                       500
Air filtration                                                        350
Animal vectors control                                                369
Cleaning technologies (robots)                                        140
Cleaning technologies robots                                            2
Control airflows in spaces & Avoid air recirculation                  568
Control airflows in spaces & avoid air recirculation                   10
Control of airflows (airtight / negative pressure)                     81
Control of airflows airtight / negative pressure                        9
Disinfection of air                                                   299
Early detection                                                       650
Ensure use of personal protection equipment                           498
Environmental parameters cont

## Mitigs


In [47]:
m = [x.label[0] for x in onto.Mitigation.instances() if len(x.label) >= 1]
len(m)

22953

In [48]:
def cleanTitle(x):
    x = str(x).replace("("," ").replace(")"," ")
    x = re.sub(' +', ' ',  str(x)).strip().capitalize()
    return x

interventions = [cleanTitle(x.label[0]) for x in onto.BP_Intervention.instances()]

In [49]:
def classify_mitigation(interventions):
    return [
        {
        "name": "classify_mitigation",
        "description": "Function used to classify mitigations from a high level perspective, based on what group they are most closely belonging to. ",
        "parameters": {
            "type": "object",
            "properties": {
                "sorted_mitigations": {
                    "type": 'array',
                    "items": {
                        "type": 'object',
                        "description": "A type of mitigation ",
                        "properties": {
                            "name" :{
                                "type": 'string', 
                                "description": 'The mitigation being considered.'
                            },
                            "category" :{
                                "type": 'string', 
                                "enum":interventions,
                                "description": 'The category of mitigation that matches the mitigation category being considered. '
                            },
                            "confidence" :{
                                "type": 'string',  
                                "enum":["High","Medium-high","Medium","Low"],
                                "description": 'Confidence that this is the correct mitigation category.'
                            }                            
                        },
                        "required": ["name",'category',"confidence"],
                    }
                },
            },
            "required": ["sorted_mitigations"],
        },
    }
]

In [61]:
RES = []
NChunks = 50
#Randomizing list
if os.path.isfile("../../support/newmitigation_classified_lv1.parquet.gzip"):
    GOOD = pd.read_parquet("../../support/newmitigation_classified_lv1.parquet.gzip")
    alreadythere = GOOD["name"].unique()
    GOOD = GOOD[GOOD.name.isin(m)]
    newRisks = [x for x in m if x not in alreadythere]
    CATS_FWK = [GOOD]
else:
    newRisks = m
    CATS_FWK = []
if len(newRisks):
    NB = len(newRisks)//NChunks + 1
    for k in range(NB)[:NBCAP]:
        try:
            QUESTION =  "The mitigations are:\n\n* "+"\n* ".join(newRisks[(k*NChunks):(k+1)*NChunks]) 
            prefix ="sh_"
            overwrite = False
            REQ = {
                "context": "Out the following list, classify the mitigations from a high level perspective.\n\n",
                "question": QUESTION,
                "functions": classify_mitigation(interventions),
                "token": TOK3N,
                "overwrite": overwrite,
                "source": "local-evalMitigationsT3.5",
                "seed" : ""
            }

            H = h.hashme(REQ["context"]+REQ["question"])
            cached = "cache/"+prefix+"_"+H+".json"
            if not os.path.isfile(cached) or overwrite:
                x = requests.post(URL+"fct/", json = REQ)
                answer = json.loads(x.text)["messages"][-1]
                h.svt(cached,answer)
            else:
                answer = h.ldt(cached)
            d = pd.DataFrame(json.loads(answer["function_call"]["arguments"])["sorted_mitigations"])
            d = d [d["category"].isin(interventions)]
            CATS_FWK.append(d)
            DFC = pd.concat(CATS_FWK).reset_index(drop=True)
            DFC = DFC.sort_values(by=["name","category"],ascending=True).drop_duplicates(subset=["name"]).reset_index(drop=True)
            DFC["name"] = DFC["name"].apply(lambda x: cleanTitle(x))
            DFC["category"] = DFC["category"].apply(lambda x: cleanTitle(x))
            DFC.to_parquet("../../support/newmitigation_classified_lv1.parquet.gzip",compression="gzip")
            print("Done:",k)
        except:
            print("error")
            pass
else:
    print("All mitigations covered")

Done: 0
Done: 1
error
Done: 3
Done: 4
Done: 5
Done: 6


In [60]:
import pandas as pd
DFC = pd.read_parquet("../../support/newmitigation_classified_lv1.parquet.gzip")
print(len(DFC),"items listed")
DFC.groupby(["category"]).name.count()


22614 items listed


category
Absorb resource shortages                                           314
Absorb surge in service demande                                     499
Air filtration                                                      350
Animal vectors control                                              369
Cleaning technologies robots                                        142
Control airflows in spaces & avoid air recirculation                578
Control of airflows airtight / negative pressure                     93
Disinfection of air                                                 298
Early detection                                                     650
Ensure use of personal protection equipment                         498
Environmental parameters control                                   1986
Facilitate transparent communication                                992
Greenery                                                            420
Greywater management                                   

# Adding CAO Classification

In [52]:
RISKS = [x.label[0] for x in onto.Risk.instances() if len(x.label) >= 1]
len(RISKS)


21166

In [36]:
DICTS_CAO = {}

DICTS_CAO["Structure"] = ["Settlement","Biodiversity","Air","Soil","Water","Communication Network", "Water Cycle", "Energy Cycle","Mobility Network","Nature","Dwelling","Buildings/Blocks","Neighbourhoods/Districts","City/Metropolis","Public Space"," Land Use"]
DICTS_CAO["Interactions"] = ["Living","Working","Shopping","Transport","Health","Education","Arts","Sports","Security","Wealth production", "Wealth Distribution","Commerce/Trade","Finances", "Competitiveness", "Entrepreneurship","Culture/Diversity","Social Expression","Heritage", "Tools&Apps","Open data", "Data in/out", "Performance (equity, resilience, self-sufficiency)"]
DICTS_CAO["Society"] = ["Person", "Family","Visitors","Organizations","Business","Participation","Capacity Development", "Leadership","Visions&Priorities", "Laws&Regulations","Accountability"]
1


1

In [37]:
DESC_Step1 = "'Structures' contains Environment (Settlements, Biodiversity, Air, Soil, Water), Infrastructure (communication networks, water, energy, matter cycle, mobility,), and Build domain (Dwelling, housing, land use, public spaces). "
DESC_Step1 += "'Interactions' contains Functions, Economy, Culture and Information, focusing on companies, economic sphere, societal, social, economic and information topics. "
DESC_Step1 += "'Society' contains Citizens and Government (inc Persons, familities, organizations, businesses, leadership, vision, laws and regulation, accountability)."
DESC_Step1 = "Use 'Structures' if the risk is in the physical world, 'Interactions' if it is about 'societal, social, economic and information' aspects and Society if talking about governance. "

In [38]:
def classifFwk_step1(lst):
    return [
        {
        "name": "classify_risks_framework",
        "description": "Function used to classify risks according to the evaluation framework, based on what they  impact the most . ",
        "parameters": {
            "type": "object",
            "properties": {
                "sorted_risks": {
                    "type": 'array',
                    "items": {
                        "type": 'object',
                        "description": "A type of risk ",
                        "properties": {
                            "name" :{
                                "type": 'string', 
                                "description": 'The risk being considered.'
                            },
                            "category" :{
                                "type": 'string', 
                                "enum":lst,
                                "description": 'The category of risk that matches the risk category being considered. '
                            },
                            "confidence" :{
                                "type": 'string',  
                                "enum":["High","Medium-high","Medium","Low"],
                                "description": 'Confidence that this is the correct risk category.'
                            }                            
                        },
                        "required": ["name",'category',"confidence"],
                    }
                },
            },
            "required": ["sorted_risks"],
        },
    }
]

In [39]:
CAO_Cats = list(DICTS_CAO.keys())

In [40]:
RES = []
NChunks = 50
#Randomizing list
if os.path.isfile("../../support/newCAO_classified_lv1.parquet.gzip"):
    GOOD = pd.read_parquet("../../support/newCAO_classified_lv1.parquet.gzip")
    alreadythere = GOOD["name"].unique()
    GOOD = GOOD[GOOD.name.isin(RISKS)]
    newRisks = [x for x in RISKS if x not in alreadythere]
    CATS_FWK = [GOOD]
else:
    newRisks = RISKS
    CATS_FWK = []
if len(newRisks):
    NB = len(newRisks)//NChunks + 1
    for k in range(NB)[:NBCAP]:
        try:
            QUESTION =  "The risks are:\n\n* "+"\n* ".join(newRisks[(k*NChunks):(k+1)*NChunks]) 
            prefix ="sh_"
            overwrite = False
            REQ = {
                "context": DESC_Step1+"\n\nOut the following list, classify the risks from a high level perspective.\n\n",
                "question": QUESTION,
                "functions": classifFwk_step1(CAO_Cats),
                "token": TOK3N,
                "overwrite": overwrite,
                "source": "local-evalMitigationsT3.5",
                "seed" : ""
            }

            H = h.hashme(REQ["context"]+REQ["question"])
            cached = "cache/"+prefix+"_"+H+".json"
            if not os.path.isfile(cached) or overwrite:
                x = requests.post(URL+"fct/", json = REQ)
                answer = json.loads(x.text)["messages"][-1]
                h.svt(cached,answer)
            else:
                answer = h.ldt(cached)
            d = pd.DataFrame(json.loads(answer["function_call"]["arguments"])["sorted_risks"])
            d = d [d["category"].isin(CAO_Cats)]
            CATS_FWK.append(d)
            DFC = pd.concat(CATS_FWK).reset_index(drop=True)
            DFC.to_parquet("../../support/newCAO_classified_lv1.parquet.gzip",compression="gzip")
            print("Done:",k)
        except:
            print("error")
            pass
else:
    print("All Risks CAO covered")

error
Done: 1


In [41]:
DFC = pd.read_parquet("../../support/newCAO_classified_lv1.parquet.gzip")
DFC.groupby(["category"]).name.count()

category
Interactions    7205
Society         5616
Structure       8295
Name: name, dtype: int64

In [42]:
OVERARCH = "You are a worldwide epidemiologist. You are studying the spread of contagious diseases in green building neighbourhoods (for example of sars-cov-2, which is the COVID)."

In [43]:
DFC = pd.read_parquet("../../support/newCAO_classified_lv1.parquet.gzip")
CATs = list(DFC.category.unique())
if "subcategory" not in DFC.columns:
    DFC["subcategory"] = None
if "confidence_lv2" not in DFC.columns:
    DFC["confidence_lv2"] = None
for CAT in CATs:
    #print("Doing",CAT)
    df = DFC[DFC.category == CAT]
    DONE = list(df[~df.subcategory.isna()].name.unique())
    df = df[~df.name.isin(DONE)]
    subtechs = DICTS_CAO[CAT] # classify_technology(CategoriesOfTechs)
    #print(subtechs)
    newRisks = list(df.name)
    RES = []
    NChunks = 50

    if len(newRisks):
        #print(len(newRisks))
        newRisks = [str(x) for x in newRisks]
        NB = len(newRisks)//NChunks +1
        for k in range(NB)[:NBCAP]:
            try:
                QUESTION =  "The risks are:\n\n* "+"\n* ".join(newRisks[(k*NChunks):(k+1)*NChunks]) 
                prefix ="CAO_risks_"
                overwrite = False
                REQ = {
                    "context": OVERARCH+"\nOut the following list, classify the risks from a high level perspective.\n\n",
                    "question": QUESTION,
                    "functions": classifFwk_step1(subtechs),
                    "token": TOK3N,
                    "overwrite": overwrite,
                    "source": "local-evalTechsT3.5",
                    "seed" : ""
                }

                H = h.hashme(REQ["context"]+REQ["question"])
                cached = "cache/"+prefix+"_"+H+".json"
                if not os.path.isfile(cached) or overwrite:
                    x = requests.post(URL+"fct/", json = REQ)
                    answer = json.loads(x.text)["messages"][-1]
                    h.svt(cached,answer)
                else:
                    answer = h.ldt(cached)
                d = pd.DataFrame(json.loads(answer["function_call"]["arguments"])["sorted_risks"])
                d = d [d["category"].isin(subtechs)]
                d.columns = ["name","subcategory","confidence_lv2"]
                for ix, row in d.iterrows():
                    DFC.loc[DFC.name == row["name"], "subcategory"] = row["subcategory"]
                    DFC.loc[DFC.name == row["name"], "confidence_lv2"] = row["confidence_lv2"]
                DFC.to_parquet("../../support/newCAO_classified_lv1.parquet.gzip",compression="gzip")
                print("Done:",k)
            except:
                print("error")
                pass

Done: 0
Done: 1
Done: 2
Done: 3
Done: 4
Done: 5
Done: 6
Done: 7
Done: 8
Done: 9
Done: 0
Done: 1
Done: 2
Done: 3
Done: 4
Done: 5
Done: 0


In [44]:
import pandas as pd
DFC = pd.read_parquet("../../support/newCAO_classified_lv1.parquet.gzip")

In [45]:
DFC[~DFC.subcategory.isna()].groupby(["category","subcategory"]).name.count().to_frame()

name
category     subcategory                                             
Interactions Arts                                                  32
             Business                                               2
             Commerce/Trade                                       208
             Competitiveness                                       10
             Culture/Diversity                                     30
             Data in/out                                          162
             Education                                            115
             Entrepreneurship                                      13
             Finances                                              36
             Health                                              5121
             Heritage                                               2
             Leadership                                             1
             Living                                               177
             Open data                                             75
             Organizations                                          3
             Performance (equity, resilience, self-sufficiency)    29
             Security                                             196
             Shopping                                              10
             Social Expression                                    205
             Sports                                                 9
             Tools&Apps                                           220
             Transport                                            172
             Visions&Priorities                                     3
             Wealth Distribution                                  200
             Wealth production                                     65
             Working                                              108
Society      Accountability                                        67
             Biodiversity                                           1
             Business                                             499
             Capacity Development                                 409
             Family                                               202
             Health                                                 1
             Laws&Regulations                                     423
             Leadership                                           411
             Organizations                                       1198
             Participation                                        239
             Person                                              1264
             Visions&Priorities                                   491
             Visitors                                             181
Structure    Air                                                 1597
             Biodiversity                                         222
             Buildings/Blocks                                     999
             City/Metropolis                                      122
             Communication Network                                890
             Dwelling                                             706
             Energy Cycle                                         287
             Laws&Regulations                                       3
             Mobility Network                                     287
             Nature                                               430
             Neighbourhoods/Districts                             229
             Person                                                 1
             Public Space                                         351
             Settlement                                           828
             Soil                                                  84
             Water                                                632
             Water Cycle                                          205
     

In [62]:
A = onto.StakeholderGroup
B = list(A.instances())[0]
print(B.label)
for C in list(B.has_GroupStakeholder)[:5]:
    print([x.label for x in C.has_StakeholderSubgroup])

['Health organisations']
[['Healthcare'], ['Medical and healthcare personnel']]
[['Public health agencies'], ['Nan'], ['Public health authorities']]
[['Medical and healthcare personnel'], ['Nan']]
[['Healthcare'], ['Medical and healthcare personnel']]
[['Medical and healthcare personnel'], ['Nan']]
